# 🌲 Tree-Based Models in Prescriptive Analytics
## ISOM 839: From Prediction to Optimization

**Tonight's Journey:**
```
Better Predictions → Better Decisions → Better Business Outcomes
```

**Learning Objectives:**
1. Understand why ML model choice impacts optimization results
2. Master Random Forests and Gradient Boosting for demand prediction
3. Integrate tree-based models with Gurobi optimization
4. Critically evaluate prediction-optimization trade-offs

---

## 🎯 The Central Question

**Last week:** You built a price optimization model using Linear Regression

**Tonight:** What if we could predict demand MORE accurately? Would our optimal prices change?

### The Prescriptive Analytics Pipeline

```
┌─────────────────┐      ┌──────────────────┐      ┌─────────────────┐
│  Historical     │      │   ML Model       │      │  Optimization   │
│  Data           │─────▶│  (Prediction)    │─────▶│  Model          │
│                 │      │                  │      │  (Decision)     │
└─────────────────┘      └──────────────────┘      └─────────────────┘
     Descriptive            Predictive              Prescriptive
```

**Key Insight:** The quality of your predictions directly affects the quality of your decisions!

---

In [ ]:
# Setup - Run this first!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Optimization Libraries
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr

# Visualization
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries loaded successfully!")
print(f"🔹 Gurobi version: {gp.gurobi.version()}")

---
# Part 1: Why Model Choice Matters

## 📊 Load the Avocado Data (Quick Review)

Let's start with the same dataset you've been working with.

In [ ]:
# Load HAB avocado data
data_url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/price_optimization/"
avocado = pd.read_csv(data_url+"HAB_data_2015to2022.csv")
avocado["date"] = pd.to_datetime(avocado["date"])
avocado = avocado.sort_values(by="date")

regions = [
    "Great_Lakes", "Midsouth", "Northeast", "Northern_New_England",
    "SouthCentral", "Southeast", "West", "Plains"
]
df = avocado[avocado.region.isin(regions)]

print(f"📦 Dataset loaded: {len(df):,} observations")
print(f"📅 Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"🌎 Regions: {len(regions)}")
print(f"\n🎯 Our goal: Predict 'units_sold' using 'price', 'region', 'year', 'peak'")

df[['date', 'region', 'price', 'units_sold', 'year', 'peak']].head()

In [ ]:
# Prepare features and target
X = df[["region", "price", "year", "peak"]]
y = df["units_sold"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42
)

print(f"✅ Training set: {len(X_train):,} samples")
print(f"✅ Testing set: {len(X_test):,} samples")

---
## 🌳 Section 1: Decision Trees - The Building Block

### What is a Decision Tree?

A decision tree is like a flowchart that asks yes/no questions about your data:

```
                    Is price < $1.50?
                    /              \
                  YES              NO
                  /                 \
         Is peak season?        Is region = West?
           /      \                /          \
         YES      NO             YES          NO
          |        |              |            |
       Predict   Predict       Predict     Predict
       3.2M      2.1M          1.8M        1.5M
```

### Advantages:
- ✅ Easy to interpret ("If price is low AND it's peak season, then...")
- ✅ Handles non-linear relationships automatically
- ✅ No need to normalize features

### Limitations:
- ❌ **High variance** - small changes in data can dramatically change the tree
- ❌ **Overfitting** - can memorize training data rather than learn patterns
- ❌ **Unstable** - not reliable for making business decisions alone

### The Solution: Ensemble Methods 🌲🌲🌲

**Core idea:** Instead of asking ONE expert, ask MANY experts and combine their answers!

In [ ]:
# Build preprocessing pipeline (same as before)
feat_transform = make_column_transformer(
    (OneHotEncoder(drop="first"), ["region"]),
    (StandardScaler(), ["price", "year"]),
    ("passthrough", ["peak"]),
    verbose_feature_names_out=False,
    remainder='drop'
)

# Train a single decision tree
tree_pipeline = make_pipeline(feat_transform, DecisionTreeRegressor(max_depth=10, random_state=42))
tree_pipeline.fit(X_train, y_train)

# Evaluate
tree_pred = tree_pipeline.predict(X_test)
tree_r2 = r2_score(y_test, tree_pred)
tree_mae = mean_absolute_error(y_test, tree_pred)

print("🌳 Single Decision Tree Performance:")
print(f"   R² Score: {tree_r2:.4f}")
print(f"   Mean Absolute Error: {tree_mae:.4f} million avocados")
print(f"\n💡 Interpretation: On average, our predictions are off by {tree_mae*1000000:,.0f} avocados")

### 💭 Discussion Question:

**Q:** Is an R² of ~0.85 "good enough" for business decisions involving millions of dollars?

**Think about:**
- What's the cost of overestimating demand? (wasted avocados)
- What's the cost of underestimating demand? (lost sales)
- Could we do better?

---

## 🌲🌲🌲 Section 2: Random Forest - Wisdom of the Crowd

### The Core Idea

**Question:** Would you rather ask:
- ONE financial analyst about a stock price?
- 100 financial analysts, then average their predictions?

**Random Forest** = Build many decision trees and average their predictions

### How It Works:

1. **Bootstrap Sampling** (Bagging):
   - Create 100 different training datasets by randomly sampling WITH replacement
   - Each tree sees a slightly different view of the data

2. **Random Feature Selection**:
   - At each split, only consider a random subset of features
   - This decorrelates the trees (makes them more independent)

3. **Aggregation**:
   - For regression: **Average** all tree predictions
   - For classification: **Majority vote**

### Why This Works:

**Individual tree:** High variance, prone to overfitting  
**Average of many trees:** Variance cancels out! More stable predictions.

**Mathematical intuition:**
- If each tree has error $\epsilon$
- Average of $n$ independent trees has error $\epsilon/\sqrt{n}$
- More trees = More stable predictions

### Key Hyperparameters:

```python
RandomForestRegressor(
    n_estimators=100,      # Number of trees (more is usually better)
    max_depth=None,        # How deep each tree can grow
    min_samples_split=2,   # Minimum samples to split a node
    max_features='sqrt',   # Features to consider at each split
    random_state=42        # For reproducibility
)
```

In [ ]:
# Train Random Forest
rf_pipeline = make_pipeline(
    feat_transform, 
    RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
)
rf_pipeline.fit(X_train, y_train)

# Evaluate
rf_pred = rf_pipeline.predict(X_test)
rf_r2 = r2_score(y_test, rf_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)

print("🌲🌲🌲 Random Forest Performance:")
print(f"   R² Score: {rf_r2:.4f}")
print(f"   Mean Absolute Error: {rf_mae:.4f} million avocados")
print(f"\n📈 Improvement over single tree:")
print(f"   R² improved by: {(rf_r2 - tree_r2):.4f} ({(rf_r2/tree_r2 - 1)*100:+.1f}%)")
print(f"   MAE reduced by: {(tree_mae - rf_mae):.4f} million ({(1 - rf_mae/tree_mae)*100:.1f}% reduction)")

In [ ]:
# Visualize the improvement
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Decision Tree predictions
axes[0].scatter(y_test, tree_pred, alpha=0.4, edgecolors='black', linewidth=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Sales (millions)', fontsize=11)
axes[0].set_ylabel('Predicted Sales (millions)', fontsize=11)
axes[0].set_title(f'Decision Tree (R² = {tree_r2:.3f})', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Random Forest predictions
axes[1].scatter(y_test, rf_pred, alpha=0.4, edgecolors='black', linewidth=0.5, color='green')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Sales (millions)', fontsize=11)
axes[1].set_ylabel('Predicted Sales (millions)', fontsize=11)
axes[1].set_title(f'Random Forest (R² = {rf_r2:.3f})', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Notice: Random Forest predictions are more tightly clustered around the red line!")

### 🔍 Feature Importance

One of the best features of Random Forest: **interpretability through feature importance**

- Measures how much each feature reduces prediction error across all trees
- Helps answer: "What REALLY drives avocado demand?"

In [ ]:
# Extract feature importance from the Random Forest model
rf_model = rf_pipeline.named_steps['randomforestregressor']
feature_names = (feat_transform.get_feature_names_out() 
                if hasattr(feat_transform, 'get_feature_names_out') 
                else [f'feature_{i}' for i in range(len(rf_model.feature_importances_))])

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualize
plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df.head(10), x='Importance', y='Feature', palette='viridis')
plt.title('What Drives Avocado Demand? (Top 10 Features)', fontsize=14, fontweight='bold')
plt.xlabel('Importance Score', fontsize=12)
plt.tight_layout()
plt.show()

print("\n📊 Feature Importance Summary:")
print(importance_df.head(10).to_string(index=False))

### 💭 Pause for Discussion:

**Q1:** Look at the feature importance. Does this make business sense?

**Q2:** If you were a supplier, what ONE action would you take based on these results?

**Q3:** Random Forest improved our R² from ~0.85 to ~0.90. Is that improvement worth it?
- Consider: computational cost, interpretability, model maintenance

---

## 🚀 Section 3: Gradient Boosting - Learning from Mistakes

### Different Philosophy from Random Forest

**Random Forest:** Build trees independently, then average  
**Gradient Boosting:** Build trees sequentially, where each tree learns from the previous tree's mistakes

### The Core Algorithm:

1. **Start simple:** Make an initial prediction (often just the mean)
2. **Calculate residuals:** Find the errors (actual - predicted)
3. **Build a tree to predict the residuals:** This tree focuses on correcting mistakes
4. **Update predictions:** Add the new tree's predictions (scaled by learning rate)
5. **Repeat:** Build the next tree on the NEW residuals

### Intuitive Example:

```
Iteration 1: Predict demand = 2.5M everywhere
            Errors: Some regions off by +0.5M, others by -0.3M

Iteration 2: Build tree to predict those errors
            New prediction = 2.5M + 0.1 * (error prediction)
            
Iteration 3: Calculate NEW errors, build another tree
            New prediction = previous + 0.1 * (new error prediction)
            
...

After 100 iterations: Very accurate predictions!
```

### Why "Gradient"?

- Each tree is fitted to the **gradient** (derivative) of the loss function
- This is like gradient descent, but building trees instead of updating weights
- Mathematical optimization meets machine learning!

### Key Hyperparameters:

```python
GradientBoostingRegressor(
    n_estimators=100,      # Number of boosting stages
    learning_rate=0.1,     # How much each tree contributes (smaller = more conservative)
    max_depth=3,           # Shallow trees work better (typically 3-6)
    subsample=0.8,         # Fraction of samples for each tree (adds randomness)
    random_state=42
)
```

### Trade-off: Learning Rate vs. Number of Trees

- **Low learning rate + Many trees** = Better accuracy, but slower training
- **High learning rate + Fewer trees** = Faster, but risk overfitting
- Common practice: Start with lr=0.1 and n_estimators=100

In [ ]:
# Train Gradient Boosting
gb_pipeline = make_pipeline(
    feat_transform,
    GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
)
gb_pipeline.fit(X_train, y_train)

# Evaluate
gb_pred = gb_pipeline.predict(X_test)
gb_r2 = r2_score(y_test, gb_pred)
gb_mae = mean_absolute_error(y_test, gb_pred)

print("🚀 Gradient Boosting Performance:")
print(f"   R² Score: {gb_r2:.4f}")
print(f"   Mean Absolute Error: {gb_mae:.4f} million avocados")
print(f"\n📈 Improvement over Random Forest:")
print(f"   R² improved by: {(gb_r2 - rf_r2):.4f} ({(gb_r2/rf_r2 - 1)*100:+.1f}%)")
print(f"   MAE reduced by: {(rf_mae - gb_mae):.4f} million ({(1 - gb_mae/rf_mae)*100:.1f}% reduction)")

---
## 📊 THE BIG COMPARISON: Linear vs. Tree vs. Forest vs. Boosting

In [ ]:
# Train linear regression for comparison
lr_pipeline = make_pipeline(feat_transform, LinearRegression())
lr_pipeline.fit(X_train, y_train)
lr_pred = lr_pipeline.predict(X_test)
lr_r2 = r2_score(y_test, lr_pred)
lr_mae = mean_absolute_error(y_test, lr_pred)

# Create comparison table
comparison = pd.DataFrame({
    'Model': ['Linear Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting'],
    'R² Score': [lr_r2, tree_r2, rf_r2, gb_r2],
    'MAE (millions)': [lr_mae, tree_mae, rf_mae, gb_mae],
    'Training Time': ['Fast', 'Fast', 'Medium', 'Slow'],
    'Interpretability': ['High', 'High', 'Medium', 'Medium'],
    'Handles Non-linearity': ['No', 'Yes', 'Yes', 'Yes']
})

print("\n" + "="*80)
print(" "*25 + "MODEL COMPARISON SUMMARY")
print("="*80)
print(comparison.to_string(index=False))
print("="*80)

# Visualize performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² comparison
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
bars = axes[0].bar(comparison['Model'], comparison['R² Score'], color=colors, alpha=0.8, edgecolor='black')
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Prediction Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylim([0.7, 1.0])
axes[0].tick_params(axis='x', rotation=15)
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# MAE comparison
bars = axes[1].bar(comparison['Model'], comparison['MAE (millions)'], color=colors, alpha=0.8, edgecolor='black')
axes[1].set_ylabel('Mean Absolute Error (millions)', fontsize=12)
axes[1].set_title('Prediction Error Comparison', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=15)
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{height:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Key Takeaway: Ensemble methods (RF and GBM) consistently outperform single models!")

---
---
# 🎯 Part 2: Integration with Optimization

## The Million Dollar Question:

**"Does a better prediction model lead to better business decisions?"**

Let's find out by plugging each model into our price optimization framework!

### Reminder: The Optimization Problem

**Decision Variables:**
- $p_r$ = price per avocado in region $r$
- $x_r$ = avocados supplied to region $r$
- $s_r$ = avocados sold in region $r$
- $d_r$ = predicted demand in region $r$

**Objective:**
$$\max \sum_r (p_r \cdot s_r - c_{waste} \cdot w_r - c^r_{transport} \cdot x_r)$$

**Key Constraints:**
- Supply constraint: $\sum_r x_r = B$
- Sales relationship: $s_r \leq x_r$ and $s_r \leq d_r(p_r, \text{region}, \text{year}, \text{peak})$
- Wastage: $w_r = x_r - s_r$

**The critical link:** $d_r$ is now predicted by our ML model! Better predictions → Better optimal decisions!

---

## 🔧 Build the Optimization Model (Using Random Forest)

We'll use Gurobi Machine Learning to embed our trained Random Forest directly into the optimization model.

In [ ]:
# Optimization parameters
B = 35  # total avocados available (millions)
peak_or_not = 1  # 1 if peak season, 0 if not
year = 2022
c_waste = 0.1  # cost per wasted avocado ($)

# Transportation costs by region
c_transport = pd.Series({
    "Great_Lakes": 0.3,
    "Midsouth": 0.1,
    "Northeast": 0.4,
    "Northern_New_England": 0.5,
    "SouthCentral": 0.3,
    "Southeast": 0.2,
    "West": 0.2,
    "Plains": 0.2,
}, name='transport_cost')
c_transport = c_transport.loc[regions]

# Price bounds
a_min = 0.5  # minimum price
a_max = 2.5  # maximum price

# Delivery bounds from historical data
data = pd.concat([
    c_transport,
    df.groupby("region")["units_sold"].min().rename('min_delivery'),
    df.groupby("region")["units_sold"].max().rename('max_delivery')
], axis=1)

print("📋 Optimization Parameters:")
print(data)
print(f"\n🎯 Total supply to allocate: {B} million avocados")
print(f"📅 Year: {year}, Peak season: {'Yes' if peak_or_not else 'No'}")

In [ ]:
# Function to solve optimization with any trained model
def solve_optimization(model_pipeline, model_name):
    """
    Solve the avocado price optimization using a given ML prediction model.
    
    Args:
        model_pipeline: Trained sklearn pipeline (must include preprocessing + model)
        model_name: String name for reporting
    
    Returns:
        solution: DataFrame with optimal prices, allocations, sales, waste
        objective_value: Optimal net revenue
    """
    print(f"\n{'='*70}")
    print(f"🎯 SOLVING OPTIMIZATION WITH: {model_name}")
    print(f"{'='*70}")
    
    # Create Gurobi model
    m = gp.Model(f"Avocado_Optimization_{model_name}")
    m.setParam('OutputFlag', 0)  # Suppress Gurobi output
    
    # Decision variables
    p = gppd.add_vars(m, data, name="price", lb=a_min, ub=a_max)
    x = gppd.add_vars(m, data, name="x", lb='min_delivery', ub='max_delivery')
    s = gppd.add_vars(m, data, name="s")
    w = gppd.add_vars(m, data, name="w")
    d = gppd.add_vars(m, data, lb=-gp.GRB.INFINITY, name="demand")
    
    m.update()
    
    # Constraints
    m.addConstr(x.sum() == B, name="total_supply")
    gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, x, name="sales_limited_by_supply")
    gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, d, name="sales_limited_by_demand")
    gppd.add_constrs(m, w, gp.GRB.EQUAL, x - s, name="wastage_definition")
    
    # Create features for ML model
    feats = pd.DataFrame({
        "year": year,
        "peak": peak_or_not,
        "region": regions,
    }, index=regions)
    
    m_feats = pd.concat([feats, p], axis=1)[["region", "price", "year", "peak"]]
    
    # Add predictor constraints (this is where the magic happens!)
    pred_constr = add_predictor_constr(m, model_pipeline, m_feats, d)
    
    print(f"\n🔗 ML Model Integration:")
    pred_constr.print_stats()
    
    # Objective: maximize net revenue
    m.setObjective(
        (p * s).sum() - c_waste * w.sum() - (c_transport * x).sum(),
        gp.GRB.MAXIMIZE
    )
    
    # Solve (nonconvex due to p * s)
    m.Params.NonConvex = 2
    m.optimize()
    
    if m.status != gp.GRB.OPTIMAL:
        print(f"❌ Optimization failed with status {m.status}")
        return None, None
    
    # Extract solution
    solution = pd.DataFrame(index=regions)
    solution["Price"] = p.gppd.X
    solution["Allocated"] = x.gppd.X
    solution["Sold"] = s.gppd.X
    solution["Wasted"] = w.gppd.X
    solution["Pred_demand"] = d.gppd.X
    
    opt_revenue = m.ObjVal
    
    print(f"\n✅ OPTIMIZATION COMPLETE")
    print(f"💰 Optimal Net Revenue: ${opt_revenue:.4f} million")
    print(f"\n📊 Solution Summary:")
    print(solution.round(4))
    
    # Check ML approximation error
    max_error = np.max(pred_constr.get_error())
    print(f"\n🎯 Max ML approximation error: {max_error:.6f}")
    
    return solution, opt_revenue

print("✅ Optimization function defined!")

---
## 🏁 The Ultimate Test: Compare All Models in Optimization

In [ ]:
# Solve with Linear Regression
sol_lr, revenue_lr = solve_optimization(lr_pipeline, "Linear Regression")

In [ ]:
# Solve with Random Forest
sol_rf, revenue_rf = solve_optimization(rf_pipeline, "Random Forest")

In [ ]:
# Solve with Gradient Boosting
sol_gb, revenue_gb = solve_optimization(gb_pipeline, "Gradient Boosting")

---
## 📈 THE BIG REVEAL: Does Better Prediction = Better Decisions?

In [ ]:
# Compare optimization results
optimization_results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'Gradient Boosting'],
    'Test R²': [lr_r2, rf_r2, gb_r2],
    'Optimal Revenue ($M)': [revenue_lr, revenue_rf, revenue_gb],
    'Revenue Improvement': [
        0,
        revenue_rf - revenue_lr,
        revenue_gb - revenue_lr
    ],
    'Improvement (%)': [
        0,
        (revenue_rf / revenue_lr - 1) * 100,
        (revenue_gb / revenue_lr - 1) * 100
    ]
})

print("\n" + "="*90)
print(" "*20 + "DOES BETTER PREDICTION = BETTER DECISIONS?")
print("="*90)
print(optimization_results.to_string(index=False))
print("="*90)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Revenue comparison
colors = ['#3498db', '#2ecc71', '#f39c12']
bars = axes[0].bar(optimization_results['Model'], optimization_results['Optimal Revenue ($M)'], 
                   color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Optimal Revenue ($M)', fontsize=12, fontweight='bold')
axes[0].set_title('Optimal Revenue by Prediction Model', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=15)
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'${height:.2f}M', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Scatter: Prediction R² vs. Revenue
axes[1].scatter(optimization_results['Test R²'], optimization_results['Optimal Revenue ($M)'], 
                s=300, c=colors, alpha=0.7, edgecolors='black', linewidth=2)
for i, model in enumerate(optimization_results['Model']):
    axes[1].annotate(model, 
                    (optimization_results['Test R²'].iloc[i], optimization_results['Optimal Revenue ($M)'].iloc[i]),
                    xytext=(10, 10), textcoords='offset points', fontsize=10, fontweight='bold')
axes[1].set_xlabel('Prediction R² Score', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Optimal Revenue ($M)', fontsize=12, fontweight='bold')
axes[1].set_title('Better Predictions → Better Decisions', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate business impact
best_revenue = optimization_results['Optimal Revenue ($M)'].max()
baseline_revenue = revenue_lr
improvement_dollars = best_revenue - baseline_revenue
improvement_pct = (improvement_dollars / baseline_revenue) * 100

print(f"\n💡 KEY BUSINESS INSIGHT:")
print(f"   Using the best ML model (vs. basic linear regression) generates:")
print(f"   📈 ${improvement_dollars:.4f} million MORE in net revenue")
print(f"   📈 {improvement_pct:.2f}% improvement")
print(f"\n   Over a year (52 weeks), this could mean:")
print(f"   💰 ${improvement_dollars * 52:.2f} million additional revenue!")

## 🔍 Deep Dive: How Do Optimal Prices Differ?

Let's examine how different prediction models lead to different pricing strategies.

In [ ]:
# Compare optimal prices across models
price_comparison = pd.DataFrame({
    'Region': regions,
    'LR_Price': sol_lr['Price'].values,
    'RF_Price': sol_rf['Price'].values,
    'GB_Price': sol_gb['Price'].values
})

price_comparison['RF_vs_LR'] = price_comparison['RF_Price'] - price_comparison['LR_Price']
price_comparison['GB_vs_LR'] = price_comparison['GB_Price'] - price_comparison['LR_Price']

print("\n📊 OPTIMAL PRICE COMPARISON BY REGION:")
print(price_comparison.round(4))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(regions))
width = 0.25

bars1 = ax.bar(x - width, price_comparison['LR_Price'], width, label='Linear Reg', 
               color='#3498db', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x, price_comparison['RF_Price'], width, label='Random Forest',
               color='#2ecc71', alpha=0.8, edgecolor='black')
bars3 = ax.bar(x + width, price_comparison['GB_Price'], width, label='Gradient Boosting',
               color='#f39c12', alpha=0.8, edgecolor='black')

ax.set_xlabel('Region', fontsize=12, fontweight='bold')
ax.set_ylabel('Optimal Price ($)', fontsize=12, fontweight='bold')
ax.set_title('How Model Choice Affects Optimal Pricing Strategy', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(regions, rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💭 INTERPRETATION:")
print(f"   Different ML models capture different demand patterns")
print(f"   → Lead to different optimal pricing strategies")
print(f"   → Result in different revenue outcomes")
print(f"\n   More accurate demand prediction = More profitable price optimization!")

---
---
# 🎓 Part 3: Best Practices & Critical Thinking

## ⚖️ The Optimization Trade-off: It's Not Always About Maximum Accuracy

### Key Dimensions to Consider:

| Dimension | Linear Regression | Random Forest | Gradient Boosting |
|-----------|------------------|---------------|-------------------|
| **Prediction Accuracy** | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **Optimization Speed** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **Model Interpretability** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ |
| **Implementation Complexity** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **Maintenance Burden** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **Stakeholder Trust** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |

### Real-World Considerations:

#### 1. **Computational Constraints**
- Gurobi ML must represent tree-based models as linear constraints
- 100 trees × 10 splits per tree = 1,000+ additional constraints
- For large-scale problems, this can make optimization intractable

#### 2. **The "Good Enough" Principle**
- If Linear Regression gives 95% of the optimal revenue with 1/10th the solve time...
- Is the extra 5% worth the complexity?
- **Business question, not just technical question!**

#### 3. **Model Maintenance & Drift**
- Complex models need more frequent retraining
- Random Forest/GBM hyperparameters need tuning
- Linear regression coefficients are easier to monitor

#### 4. **Explainability to Stakeholders**
- CFO asks: "Why did we set the Northeast price at $1.85?"
- Linear Regression: "For every $0.10 increase in price, demand drops by X units"
- Random Forest: "Um... 100 trees voted and..."

---

## 🎯 Decision Framework: When to Use What?

### Use **Linear Regression** when:
- ✅ Relationships are approximately linear
- ✅ You need to explain every decision to stakeholders
- ✅ Optimization must run in real-time (< 1 second)
- ✅ Model stability over time is critical
- ✅ You have limited ML expertise on the team

**Example:** Daily pricing updates for 10,000 products across 500 stores

### Use **Random Forest** when:
- ✅ Relationships are clearly non-linear
- ✅ You have moderate computational resources
- ✅ Feature importance insights are valuable
- ✅ Stakeholders trust "data-driven black boxes"
- ✅ The accuracy gain justifies added complexity

**Example:** Weekly supply chain optimization for 50 products across 8 regions

### Use **Gradient Boosting** when:
- ✅ Maximum accuracy is paramount (competitions, high-value decisions)
- ✅ Optimization runs infrequently (monthly strategic planning)
- ✅ You have strong ML expertise and computational resources
- ✅ The business impact of small improvements is massive
- ✅ You can afford longer solve times

**Example:** Annual strategic capacity planning for a $500M business unit

---

## 🧪 Practical Exercise: The Business Case

**Scenario:** You're presenting to the VP of Supply Chain. They ask:

> "You're recommending we switch from our simple regression model to this Random Forest thing. 
> It will cost us $50K to implement and maintain annually. 
> Our current annual avocado revenue is $120 million. 
> Is it worth it?"

### Your Task: Build a Business Case

Use the optimization results to calculate:
1. **Expected annual revenue improvement**
2. **ROI (Return on Investment)**
3. **Payback period**
4. **Key risks and mitigations**

In [ ]:
# Business case calculation
weekly_improvement = revenue_rf - revenue_lr  # in millions
annual_improvement = weekly_improvement * 52  # 52 weeks
implementation_cost = 0.05  # $50K = $0.05M
annual_maintenance_cost = 0.05  # $50K = $0.05M

net_benefit_year1 = annual_improvement - implementation_cost - annual_maintenance_cost
net_benefit_steady_state = annual_improvement - annual_maintenance_cost
roi = (net_benefit_year1 / implementation_cost) * 100
payback_months = (implementation_cost / (annual_improvement / 12))

print("\n" + "="*80)
print(" "*25 + "BUSINESS CASE ANALYSIS")
print("="*80)
print(f"\n📊 REVENUE IMPACT:")
print(f"   Weekly improvement: ${weekly_improvement:.4f} million")
print(f"   Annual improvement: ${annual_improvement:.4f} million")

print(f"\n💰 COSTS:")
print(f"   Implementation (one-time): ${implementation_cost:.2f} million")
print(f"   Annual maintenance: ${annual_maintenance_cost:.2f} million")

print(f"\n📈 FINANCIAL METRICS:")
print(f"   Net benefit (Year 1): ${net_benefit_year1:.4f} million")
print(f"   Net benefit (subsequent years): ${net_benefit_steady_state:.4f} million")
print(f"   ROI: {roi:.1f}%")
print(f"   Payback period: {payback_months:.1f} months")

print(f"\n🎯 RECOMMENDATION:")
if net_benefit_year1 > 0:
    print(f"   ✅ PROCEED with Random Forest implementation")
    print(f"   ✅ Expect ${net_benefit_year1*1000:.0f}K additional profit in Year 1")
    print(f"   ✅ Project pays for itself in {payback_months:.0f} months")
else:
    print(f"   ❌ NOT RECOMMENDED - Costs exceed benefits")

print("="*80)

# Sensitivity analysis
print(f"\n🔬 SENSITIVITY ANALYSIS:")
print(f"   If improvement is 50% of predicted: ${(annual_improvement * 0.5 - implementation_cost - annual_maintenance_cost):.4f}M net benefit")
print(f"   If improvement is 150% of predicted: ${(annual_improvement * 1.5 - implementation_cost - annual_maintenance_cost):.4f}M net benefit")
print(f"\n   Even at 50% of predicted improvement, ROI is positive! ✅")

---
## 🎓 KEY TAKEAWAYS

### 1. **The Prescriptive Analytics Mindset**
- Prediction is a means to an end, not the end itself
- The goal is better **decisions**, not just better **models**
- Always connect ML metrics to business outcomes (R² → Revenue)

### 2. **Tree-Based Models Are Powerful**
- **Random Forest:** Stable, interpretable, good first choice for non-linear problems
- **Gradient Boosting:** Maximum accuracy, but requires careful tuning
- Both significantly outperform linear models on complex data

### 3. **Integration with Optimization**
- Gurobi ML seamlessly embeds sklearn models into optimization
- Tree-based models add complexity (more constraints)
- Trade-off: Prediction accuracy vs. Optimization speed

### 4. **The "Good Enough" Principle**
- Perfect is the enemy of good in production systems
- Consider: accuracy, speed, interpretability, maintenance
- **Best model ≠ Most complex model**

### 5. **Business-Driven Decisions**
- Calculate ROI before implementation
- Understand stakeholder needs (CFO vs. Data Scientist)
- Build trust through explainability

---

## 💭 Critical Thinking Questions for Discussion

### Question 1: Model Selection
**Scenario:** You're optimizing pricing for an e-commerce platform with 10,000 products. Prices update every hour based on competitor pricing and inventory levels.

- Which model would you choose: Linear Regression, Random Forest, or Gradient Boosting?
- What are the key factors in your decision?
- What might go wrong with each choice?

### Question 2: Explainability vs. Accuracy
**Scenario:** Your Gradient Boosting model achieves 95% accuracy, but the CEO asks, "Why did we set this specific price?" and you can't give a clear answer. Your Linear Regression achieves 88% accuracy but you can explain every coefficient.

- Which do you deploy to production?
- How do you handle the explainability gap?
- Are there hybrid approaches?

### Question 3: The False Precision Trap
**Scenario:** Your model predicts demand will be 2,347,891 avocados. Your optimization sets the price at $1.8374.

- Is this level of precision meaningful?
- How do you communicate uncertainty to stakeholders?
- Should you round optimal solutions?

### Question 4: Model Drift
**Scenario:** After 6 months in production, your Random Forest's accuracy has dropped from 92% to 78%. Market conditions changed (pandemic, competitor entry, etc.).

- How do you detect this in production?
- What's your retraining strategy?
- How does model complexity affect maintenance burden?

### Question 5: Ethical Considerations
**Scenario:** Your model learns that certain regions tolerate higher prices. This leads to differential pricing that could be seen as discriminatory.

- Is this optimization ethical?
- How do you balance profit maximization with fairness?
- What constraints might you add to the optimization?

---

## 🚀 Advanced Topics (For Further Exploration)

### 1. **Hyperparameter Tuning**
- Use GridSearchCV or RandomizedSearchCV
- Cross-validation for robust evaluation
- But remember: overfit to test set is still overfitting!

### 2. **XGBoost and LightGBM**
- More efficient implementations of gradient boosting
- Handle larger datasets, faster training
- Industry standard for competitions and production

### 3. **Stochastic Optimization**
- What if demand is uncertain?
- Robust optimization: optimize for worst-case
- Chance constraints: "95% probability of meeting demand"

### 4. **Multi-Stage Optimization**
- Today: predict demand, then optimize once
- Advanced: optimize pricing policy that adapts to realized demand
- Stochastic programming, dynamic programming

### 5. **Interpretable ML (SHAP, LIME)**
- Explain individual predictions from black-box models
- Build trust without sacrificing accuracy
- Bridge between complex models and stakeholders

---

## 📚 Additional Resources

### Books:
- **"The Elements of Statistical Learning"** - Hastie, Tibshirani, Friedman (Classic ML textbook)
- **"Hands-On Machine Learning"** - Aurélien Géron (Practical sklearn guide)
- **"Optimization in Operations Research"** - Ronald Rardin

### Documentation:
- Gurobi Machine Learning: https://gurobi-machinelearning.readthedocs.io/
- Scikit-learn Ensemble Methods: https://scikit-learn.org/stable/modules/ensemble.html
- Gurobipy-pandas: https://github.com/Gurobi/gurobipy-pandas

### Papers:
- Breiman, L. (2001). "Random Forests." *Machine Learning*
- Friedman, J. H. (2001). "Greedy Function Approximation: A Gradient Boosting Machine." *Annals of Statistics*

### Kaggle Competitions (for practice):
- Store Sales Forecasting
- Demand Prediction Challenges
- Any regression problem with business context

---

## ✅ Quick Review Quiz

**Test your understanding:**

1. **True or False:** Random Forest always outperforms Gradient Boosting.
   - *Answer: False. GB often achieves higher accuracy, but RF can be better with limited tuning or parallel processing needs.*

2. **What is the key difference between Random Forest and Gradient Boosting?**
   - *Answer: RF builds trees independently in parallel; GB builds trees sequentially where each learns from previous errors.*

3. **Why might you choose Linear Regression over Random Forest in prescriptive analytics?**
   - *Answer: Speed, interpretability, stakeholder trust, easier maintenance, good enough accuracy for business needs.*

4. **How does Gurobi ML embed tree-based models into optimization?**
   - *Answer: Converts decision tree logic into linear constraints that Gurobi can solve.*

5. **What is feature importance and why does it matter for business?**
   - *Answer: Measures which features most influence predictions; helps prioritize data collection and informs business strategy.*

6. **When is high prediction accuracy NOT the right goal?**
   - *Answer: When stakeholders need explainability, when optimization speed matters, when maintenance burden exceeds benefit, when "good enough" suffices.*

---

## 🎯 Final Thoughts: The Prescriptive Analytics Mindset

```
❌ Wrong approach:
   "I built the most accurate model! R² = 0.99!"

✅ Right approach:
   "I improved revenue by $2M/year using a model stakeholders trust,
    solves in under 5 seconds, and maintenance is straightforward.
    R² = 0.92."
```

### Remember:

1. **Start simple, add complexity only when justified**
2. **Connect every metric to business value**
3. **Build trust through transparency**
4. **Production-ready > Kaggle-winner**
5. **Better decisions > Better predictions**

---

## 📬 Thank You!

**Tonight you learned:**
- ✅ How tree-based ensemble methods work
- ✅ When to use Random Forest vs. Gradient Boosting vs. Linear Regression
- ✅ How to integrate ML models into optimization frameworks
- ✅ How to evaluate models through a business lens
- ✅ The critical thinking needed for prescriptive analytics

**Next steps:**
- Practice with your own datasets
- Experiment with hyperparameter tuning
- Try XGBoost and LightGBM
- Build a complete end-to-end prescriptive analytics project

### Questions? Let's discuss! 🤔

---

*"The goal is to turn data into information, and information into insight, and insight into better decisions."*  
— Carly Fiorina, Former CEO of HP